In [4]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
wine = load_wine()
X, y = wine.data, wine.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import joblib, matplotlib.pyplot as plt, os
def train_evaluate_save(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='macro'),
        'recall': recall_score(y_test, y_pred, average='macro'),
        'f1': f1_score(y_test, y_pred, average='macro')
    }
    os.makedirs('../models', exist_ok=True)
    joblib.dump(model, f'../models/{model_name}.pkl')
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=wine.target_names)
    disp.plot()
    plt.savefig(f'../results/{model_name}_confusion_matrix.png')
    plt.close()
    return metrics

In [9]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000)
lr_metrics = train_evaluate_save(lr, 'logistic_regression')

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_metrics = train_evaluate_save(rf, 'random_forest')

In [11]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', random_state=42)
svm_metrics = train_evaluate_save(svm, 'svm')

In [14]:
import pandas as pd

metrics_df = pd.DataFrame([lr_metrics, rf_metrics, svm_metrics], index=['Logistic Regression', 'Random Forest', 'SVM'])

metrics_df.to_csv('../results/model_comparison.csv')

In [16]:
import mlflow
import mlflow.sklearn
mlflow.set_experiment("Wine Classification")
mlflow.set_tracking_uri("http://localhost:5000")

2025/09/18 12:39:29 INFO mlflow.tracking.fluent: Experiment with name 'Wine Classification' does not exist. Creating a new experiment.


In [18]:
def train_log_mlflow(model, model_name, params, experiment_name="Wine_Classification"):
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        metrics = train_evaluate_save(model, model_name)
        for metric, value in metrics.items():
            mlflow.log_metric(metric, value)
        mlflow.log_artifact(f'../results/{model_name}_confusion_matrix.png')
        mlflow.sklearn.log_model(model, "model")
        return mlflow.active_run().info.run_id

In [19]:
lr_params1 = {'max_iter': 1000, 'C': 0.1}
lr_run1_id = train_log_mlflow(LogisticRegression(**lr_params1), 'lr_run1', lr_params1)  # Acc: 0.944
lr_params2 = {'max_iter': 1000, 'C': 1.0}
lr_run2_id = train_log_mlflow(LogisticRegression(**lr_params2), 'lr_run2', lr_params2)  # Acc: 1.0
lr_params3 = {'max_iter': 1000, 'C': 10.0}
lr_run3_id = train_log_mlflow(LogisticRegression(**lr_params3), 'lr_run3', lr_params3)  # Acc: 1.0

2025/09/18 12:43:52 INFO mlflow.tracking.fluent: Experiment with name 'Wine_Classification' does not exist. Creating a new experiment.
2025/09/18 12:43:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 12:44:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run lr_run1 at: http://localhost:5000/#/experiments/201533867294901606/runs/9fba626cb10a4c3bba90e39301ea6cdf
🧪 View experiment at: http://localhost:5000/#/experiments/201533867294901606


2025/09/18 12:44:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 12:44:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run lr_run2 at: http://localhost:5000/#/experiments/201533867294901606/runs/6c5bf7b9ddb94140af8435c5e6ac6c7f
🧪 View experiment at: http://localhost:5000/#/experiments/201533867294901606


2025/09/18 12:44:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 12:45:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run lr_run3 at: http://localhost:5000/#/experiments/201533867294901606/runs/46db1eda2cef45bdbb891bda573ab576
🧪 View experiment at: http://localhost:5000/#/experiments/201533867294901606


In [20]:
rf_params1 = {'n_estimators': 50, 'random_state': 42}
rf_run1_id = train_log_mlflow(RandomForestClassifier(**rf_params1), 'rf_run1', rf_params1)  
rf_params2 = {'n_estimators': 100, 'random_state': 42}
rf_run2_id = train_log_mlflow(RandomForestClassifier(**rf_params2), 'rf_run2', rf_params2)  
rf_params3 = {'n_estimators': 200, 'random_state': 42}
rf_run3_id = train_log_mlflow(RandomForestClassifier(**rf_params3), 'rf_run3', rf_params3)

2025/09/18 12:46:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 12:46:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rf_run1 at: http://localhost:5000/#/experiments/201533867294901606/runs/0b775fc00c8d4337bf94232f90a996dc
🧪 View experiment at: http://localhost:5000/#/experiments/201533867294901606


2025/09/18 12:46:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 12:46:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rf_run2 at: http://localhost:5000/#/experiments/201533867294901606/runs/f576221ec80f4674b6bdc5047a3d016a
🧪 View experiment at: http://localhost:5000/#/experiments/201533867294901606


2025/09/18 12:46:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 12:46:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rf_run3 at: http://localhost:5000/#/experiments/201533867294901606/runs/ea9be25ee30a4c8b8236e389d2ee668c
🧪 View experiment at: http://localhost:5000/#/experiments/201533867294901606


In [21]:
svm_params1 = {'kernel': 'linear', 'C': 0.1, 'random_state': 42}
svm_run1_id = train_log_mlflow(SVC(**svm_params1), 'svm_run1', svm_params1)  
svm_params2 = {'kernel': 'linear', 'C': 1.0, 'random_state': 42}
svm_run2_id = train_log_mlflow(SVC(**svm_params2), 'svm_run2', svm_params2)  
svm_params3 = {'kernel': 'linear', 'C': 10.0, 'random_state': 42}
svm_run3_id = train_log_mlflow(SVC(**svm_params3), 'svm_run3', svm_params3)

2025/09/18 12:47:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 12:47:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run svm_run1 at: http://localhost:5000/#/experiments/201533867294901606/runs/7180d333f4a8422a9e51272fde0c32b7
🧪 View experiment at: http://localhost:5000/#/experiments/201533867294901606


2025/09/18 12:47:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 12:47:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run svm_run2 at: http://localhost:5000/#/experiments/201533867294901606/runs/67afc40efac646b39d1c4dcfd1c0a39a
🧪 View experiment at: http://localhost:5000/#/experiments/201533867294901606


2025/09/18 12:47:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 12:47:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run svm_run3 at: http://localhost:5000/#/experiments/201533867294901606/runs/f3d50c8035724c1fb02af94783e8e5bb
🧪 View experiment at: http://localhost:5000/#/experiments/201533867294901606


In [22]:
run_id = lr_run2_id
mlflow.register_model(f"runs:/{run_id}/model", "WineClassifier")

Successfully registered model 'WineClassifier'.
2025/09/18 13:04:09 WARNING mlflow.tracking._model_registry.fluent: Run with id 6c5bf7b9ddb94140af8435c5e6ac6c7f has no artifacts at artifact path 'model', registering model based on models:/m-64fb3632ad784b318c3ce1d0aff18aec instead
2025/09/18 13:04:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WineClassifier, version 1
Created version '1' of model 'WineClassifier'.


<ModelVersion: aliases=[], creation_timestamp=1758182649095, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1758182649095, metrics=None, model_id=None, name='WineClassifier', params=None, run_id='6c5bf7b9ddb94140af8435c5e6ac6c7f', run_link='', source='models:/m-64fb3632ad784b318c3ce1d0aff18aec', status='READY', status_message=None, tags={}, user_id='', version='1'>